In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_test_data = pd.concat([train_data, test_data])

In [4]:
train_test_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0.0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0.0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0.0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
train_test_data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [6]:
train_test_data.shape

(1309, 12)

In [7]:
train_test_data['Cabin'] = train_test_data['Cabin'].str[:1]

In [8]:
object_columns = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

for column in object_columns:
    train_test_data[column] = pd.factorize(train_test_data[column])[0]

In [9]:
train_test_data['Cabin'] = train_test_data['Cabin'].replace(0, np.nan)

In [10]:
train_test_data['Cabin'].fillna(train_test_data.groupby('Pclass')['Cabin'].transform('median'), inplace=True)

In [11]:
train_test_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,0,0,22.0,1,0,0,7.2500,-1.0,0
1,2,1.0,1,1,1,38.0,1,0,1,71.2833,3.0,1
2,3,1.0,3,2,1,26.0,0,0,2,7.9250,-1.0,0
3,4,1.0,1,3,1,35.0,1,0,3,53.1000,3.0,0
4,5,0.0,3,4,0,35.0,0,0,4,8.0500,-1.0,0
5,6,0.0,3,5,0,NaN,0,0,5,8.4583,-1.0,2
6,7,0.0,1,6,0,54.0,0,0,6,51.8625,1.0,0
7,8,0.0,3,7,0,2.0,3,1,7,21.0750,-1.0,0
8,9,1.0,3,8,1,27.0,0,2,8,11.1333,-1.0,0
9,10,1.0,2,9,1,14.0,1,0,9,30.0708,-1.0,1


In [12]:
train_data = train_test_data[~train_test_data['Survived'].isnull()]
test_data = train_test_data[train_test_data['Survived'].isnull()]

In [13]:
ftr_train = train_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1)
target = train_data['Survived']

In [14]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr_train, target, test_size=0.3, random_state=2022)

In [57]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

bayesian_params = {
    'max_depth': (2, 8), 
    'num_leaves': (2, 12), 
    'min_child_samples': (1, 50), 
    'min_child_weight':(1, 20),
    'subsample':(0.5, 1.0),
    'max_bin':(10, 500),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

In [63]:
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":100, "learning_rate":0.02,
        'max_depth': int(round(max_depth)),
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0)
    }
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 50, 
                early_stopping_rounds= 50)
    valid_proba = lgb_model.predict_proba(valid_x)[:, 1]
    roc_auc = roc_auc_score(valid_y, valid_proba)
    
    return roc_auc   

In [64]:
lgbB0 = BayesianOptimization(f=lgb_roc_eval, pbounds=bayesian_params, random_state=2022)
lgbB0.maximize(init_points=5, n_iter=25)

|   iter    |  target   |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
[50]	training's auc: 0.844591	training's binary_logloss: 0.546484	valid_1's auc: 0.838869	valid_1's binary_logloss: 0.547006
[100]	training's auc: 0.849027	training's binary_logloss: 0.507979	valid_1's auc: 0.849851	valid_1's binary_logloss: 0.508576
|  1        |  0.8488   |  14.59    |  4.994    |  6.556    |  1.95     |  8.854    |  24.35    |  8.977    |  0.8237   |


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument ins

[50]	training's auc: 0.820232	training's binary_logloss: 0.603963	valid_1's auc: 0.852083	valid_1's binary_logloss: 0.595709
|  2        |  0.8521   |  449.5    |  6.327    |  41.74    |  16.72    |  10.34    |  47.85    |  3.681    |  0.7474   |
[50]	training's auc: 0.85483	training's binary_logloss: 0.515903	valid_1's auc: 0.841875	valid_1's binary_logloss: 0.524996


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument ins

[100]	training's auc: 0.856126	training's binary_logloss: 0.47061	valid_1's auc: 0.848512	valid_1's binary_logloss: 0.481535
|  3        |  0.8485   |  176.4    |  5.717    |  48.9     |  2.832    |  9.442    |  14.63    |  2.987    |  0.8762   |
[50]	training's auc: 0.831159	training's binary_logloss: 0.541099	valid_1's auc: 0.841042	valid_1's binary_logloss: 0.542074
[100]	training's auc: 0.838143	training's binary_logloss: 0.503705	valid_1's auc: 0.847381	valid_1's binary_logloss: 0.504477
|  4        |  0.8474   |  19.15    |  5.142    |  43.36    |  8.388    |  4.122    |  23.76    |  5.647    |  0.6747   |


...

[50]	training's auc: 0.877985	training's binary_logloss: 0.467363	valid_1's auc: 0.853244	valid_1's binary_logloss: 0.488767
[100]	training's auc: 0.897188	training's binary_logloss: 0.408446	valid_1's auc: 0.858006	valid_1's binary_logloss: 0.446883
|  27       |  0.858    |  495.0    |  5.025    |  9.423    |  6.363    |  11.21    |  0.5122   |  4.021    |  0.9055   |


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	training's auc: 0.87439	training's binary_logloss: 0.48508	valid_1's auc: 0.860714	valid_1's binary_logloss: 0.498373
|  28       |  0.8607   |  496.6    |  6.683    |  1.105    |  2.507    |  8.758    |  6.916    |  1.604    |  0.713    |


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument ins

[50]	training's auc: 0.877644	training's binary_logloss: 0.46891	valid_1's auc: 0.858304	valid_1's binary_logloss: 0.4847
|  29       |  0.8598   |  489.9    |  6.518    |  19.65    |  2.372    |  8.881    |  3.623    |  0.0364   |  0.8662   |
[50]	training's auc: 0.883235	training's binary_logloss: 0.458086	valid_1's auc: 0.851429	valid_1's binary_logloss: 0.485174
[100]	training's auc: 0.898777	training's binary_logloss: 0.396231	valid_1's auc: 0.861696	valid_1's binary_logloss: 0.440509
|  30       |  0.8617   |  497.1    |  4.034    |  3.437    |  3.37     |  11.01    |  0.5429   |  3.623    |  0.9549   |


In [65]:
lgbB0.res

[{'target': 0.8487797619047619,
  'params': {'max_bin': 14.585720765804705,
   'max_depth': 4.994346865111928,
   'min_child_samples': 6.555800806806923,
   'min_child_weight': 1.949506345374979,
   'num_leaves': 8.854075942430917,
   'reg_alpha': 24.354533530104362,
   'reg_lambda': 8.976674607206252,
   'subsample': 0.8237260353716331}},
 {'target': 0.8520833333333333,
  'params': {'max_bin': 449.5119301675884,
   'max_depth': 6.326809574988113,
   'min_child_samples': 41.7363176180333,
   'min_child_weight': 16.72379330761429,
   'num_leaves': 10.335795840581405,
   'reg_alpha': 47.85264633978365,
   'reg_lambda': 3.681076325612883,
   'subsample': 0.7474188149426438}},
 {'target': 0.8485119047619047,
  'params': {'max_bin': 176.3596425478705,
   'max_depth': 5.716575957469941,
   'min_child_samples': 48.89895228264298,
   'min_child_weight': 2.8322284751214624,
   'num_leaves': 9.442062122546155,
   'reg_alpha': 14.632048710028132,
   'reg_lambda': 2.9874548342985574,
   'subsample

In [66]:
target_list = []
for result in lgbB0.res:
    target = result['target']
    target_list.append(target)

print(target_list)
print('maximum target index', np.argmax(np.array(target_list)))

[0.8487797619047619, 0.8520833333333333, 0.8485119047619047, 0.8473809523809523, 0.8523214285714287, 0.8589583333333333, 0.8520833333333333, 0.8465178571428572, 0.8410416666666667, 0.8410416666666667, 0.8520833333333333, 0.8505952380952381, 0.8520833333333333, 0.8461309523809523, 0.8505952380952381, 0.8520833333333333, 0.8523214285714287, 0.8505952380952381, 0.8520833333333333, 0.8523214285714287, 0.8627678571428571, 0.8623511904761905, 0.8554761904761905, 0.8418749999999999, 0.8532440476190475, 0.8572023809523809, 0.8580059523809522, 0.8607142857142857, 0.8597619047619047, 0.8616964285714286]
maximum target index 20


In [67]:
max_dict = lgbB0.res[np.argmax(np.array(target_list))]
print(max_dict)

{'target': 0.8627678571428571, 'params': {'max_bin': 498.764153235885, 'max_depth': 3.4584575615133377, 'min_child_samples': 3.1346651672092594, 'min_child_weight': 3.4478833396768676, 'num_leaves': 8.087614797859061, 'reg_alpha': 3.878296666808249, 'reg_lambda': 5.230946367170306, 'subsample': 0.5227621631488063}}


In [71]:
clf = LGBMClassifier(
    n_jobs=-1,
    n_estimators=50,
    learning_rate=0.02,
    max_bin=498,
    min_child_samples=3,
    min_child_weight=3,
    num_leaves=8,
    subsample=0.982,
    max_depth=3,
    reg_alpha=3.878,
    reg_lambda=5.230,
    silent=-1,
    verbose=-1
)

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=30,
       early_stopping_rounds=30)

[30]	training's auc: 0.876288	training's binary_logloss: 0.527835	valid_1's auc: 0.86247	valid_1's binary_logloss: 0.534941


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verb

LGBMClassifier(learning_rate=0.02, max_bin=498, max_depth=3,
               min_child_samples=3, min_child_weight=3, n_estimators=50,
               num_leaves=8, reg_alpha=3.878, reg_lambda=5.23, silent=-1,
               subsample=0.982, verbose=-1)

In [72]:
preds = clf.predict_proba(test_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1))[:, 1]

In [73]:
preds.tolist()

[0.2344571706330496,
 0.42062414698704864,
 0.2344571706330496,
 0.2344571706330496,
 0.4257124106999716,
 0.31939520543403027,
 0.42062414698704864,
 0.2910731626274551,
 0.4257124106999716,
 0.2344571706330496,
 0.2344571706330496,
 0.2344571706330496,
 0.6791674345407794,
 0.2344571706330496,
 0.6791674345407794,
 0.6791674345407794,
 0.2344571706330496,
 0.2344571706330496,
 0.4257124106999716,
 0.42062414698704864,
 0.2910731626274551,
 0.4472735043859007,
 0.6791674345407794,
 0.2910731626274551,
 0.6791674345407794,
 0.2344571706330496,
 0.6791674345407794,
 0.2344571706330496,
 0.396346541073757,
 0.2344571706330496,
 0.2344571706330496,
 0.2910731626274551,
 0.42062414698704864,
 0.44036930570038924,
 0.396346541073757,
 0.2344571706330496,
 0.44036930570038924,
 0.4257124106999716,
 0.2344571706330496,
 0.2910731626274551,
 0.2344571706330496,
 0.38568260853056385,
 0.2344571706330496,
 0.6791674345407794,
 0.6791674345407794,
 0.2344571706330496,
 0.38568260853056385,
 0.234

In [74]:
preds = [1 if x > 0.5 else 0 for x in preds]

In [76]:
test_data['Survived'] = preds

/var/folders/09/xdnxbw6n7bl58mxczkp01qq40000gn/T/ipykernel_18188/1324259112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['Survived'] = preds


In [77]:
test_data[['PassengerId', 'Survived']].to_csv('titanic_hyperparameter_tuning.csv', index=False)